# (D) Predictive Analytics

In [1]:
import numpy as np 
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

# Set up visualization style
plt.style.use('ggplot')
sns.set_palette("husl")

# View all columns of a dataframe
pd.set_option('display.max_columns', None)

import os

from scipy.stats import dirichlet
from scipy.stats import t

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## 01. Predicting Disease Prevalence

In [2]:
prevalence_per_100000_df = pd.read_csv('../results/prevalence_per_100000_by_region_age_year.csv')

id_cols = ['Geography', 'Age_Group', 'Disease']

train_years = [str(y) for y in range(2018, 2023)]  # 2018 to 2022
train_df = prevalence_per_100000_df[id_cols + train_years]

# Test data: year 2023
test_df = prevalence_per_100000_df[id_cols + ['2023']]

Our prediction model is of the form, 

$$
\text{y\_pred (2023, geography, age\_group, disease)} = k \times \text{trend\_factor} \times \text{disease\_prevalence\_across\_geography } \times 
$$
$$
\text{probability\_of\_disease\_in\_that\_age\_group}
$$

where k is the normalizing constant. 

### Step 1. Create Bayesian Probability Distribution of Diseases Across Age Groups

In [3]:
year_cols = ['2018', '2019', '2020', '2021', '2022']

long_df = train_df.melt(id_vars=id_cols, value_vars=year_cols, var_name='Year', value_name='Value')

In [4]:
results = []

for disease, group_df in long_df.groupby('Disease'):
    age_groups = group_df['Age_Group'].unique()
    # Collect all values per age group
    counts_per_age = []
    for age in age_groups:
        vals = group_df.loc[group_df['Age_Group']==age, 'Value'].values
        # Use the sum across council-year as pseudo-counts
        counts_per_age.append(vals.sum())
    
    counts_per_age = np.array(counts_per_age) + 1e-3  # avoid zeros
    # Dirichlet posterior
    alpha = counts_per_age
    mean_prob = alpha / alpha.sum()
    
    # Sample from Dirichlet to get uncertainty
    samples = dirichlet.rvs(alpha, size=10000)
    lower = np.percentile(samples, 2.5, axis=0)
    upper = np.percentile(samples, 97.5, axis=0)
    
    for age, p, l, u in zip(age_groups, mean_prob, lower, upper):
        results.append({
            'Disease': disease,
            'AgeGroup': age,
            'Probability': p,
            'LowerBound': l,
            'UpperBound': u
        })

In [5]:
prob_df = pd.DataFrame(results)

In [6]:
output_folder = 'disease_agegroup_plots'
os.makedirs(output_folder, exist_ok=True)

for disease, df_plot in prob_df.groupby('Disease'):
    disease_folder = os.path.join(output_folder, disease.replace('/', '_'))
    os.makedirs(disease_folder, exist_ok=True)
    
    plt.figure(figsize=(10,5))
    yerr = [np.abs(df_plot['Probability'] - df_plot['LowerBound']),
            np.abs(df_plot['UpperBound'] - df_plot['Probability'])]
    
    plt.bar(df_plot['AgeGroup'], df_plot['Probability'],
            yerr=yerr,
            capsize=5, alpha=0.7)
    
    plt.title(f"Age Group Probability Distribution for {disease}", fontsize=12)
    plt.ylabel("Probability")
    plt.xlabel("Age Group")
    plt.xticks(rotation=45)
    
    plot_path = os.path.join(disease_folder, f"{disease.replace('/', '_')}_agegroup_prob.png")
    plt.savefig(plot_path, bbox_inches='tight')
    plt.close()

In [7]:
prob_df.to_csv('../results/probability_of_disease_per_age_group.csv', index=None)

### Step 2. Create Bayesian Distribution of Diseases Across Councils

In [8]:
long_df = train_df.melt(id_vars=id_cols, value_vars=year_cols,
                        var_name='Year', value_name='Value')

results = []

for (geo, disease), group in long_df.groupby(['Geography', 'Disease']):
    values = group['Value'].values
    n = len(values)
    mean_val = np.mean(values)
    std_val = np.std(values, ddof=1)  # sample std
    dof = n - 1
    
    # 95% credible interval using t-distribution
    t_crit = t.ppf(0.975, df=dof)
    margin = t_crit * (std_val / np.sqrt(n))
    lower = mean_val - margin
    upper = mean_val + margin
    
    results.append({
        'Geography': geo,
        'Disease': disease,
        'Count': mean_val,
        'LowerBound': lower,
        'UpperBound': upper
    })

geo_disease_df = pd.DataFrame(results)

In [9]:
geo_disease_df.to_csv('../results/disease_prevalence_per_location.csv', index=None)

### Step 3. Create a Linear Regression for Each Disease For Past 5 Years To Capture Trend

In [10]:
year_cols = ['2018','2019','2020','2021','2022']
long_df = train_df.melt(id_vars=['Geography','Age_Group','Disease'], value_vars=year_cols, var_name='Year', value_name='Value')
long_df['Year'] = long_df['Year'].astype(int)

# Encode years as 1..5
year_mapping = {2018:1, 2019:2, 2020:3, 2021:4, 2022:5}
long_df['YearCode'] = long_df['Year'].map(year_mapping)

# Center YearCode
mean_X = long_df['YearCode'].mean()  # will be 3
long_df['YearC'] = long_df['YearCode'] - mean_X

In [11]:
trend_dict = {}  # (Geo, AgeGroup, Disease) -> trend_factor

for (geo, age, disease), group in long_df.groupby(['Geography','Age_Group','Disease']):
    X = group['YearC'].values.reshape(-1,1)
    y = group['Value'].values
    last_val = group.loc[group['Year']==2022, 'Value'].values[0]

    if np.all(y==0) or last_val == 0:  # avoid divide by zero
        trend_factor = 1
    else:
        model = LinearRegression().fit(X, y)
        beta_1 = model.coef_[0]   # slope
        trend_factor = 1 + beta_1 / last_val  # multiplicative adjustment
        # print(f"{geo}, {age}, {disease} -> last_val: {last_val:.2f}, beta_1: {beta_1:.2f}, trend_factor: {trend_factor:.3f}")
        
    trend_dict[(geo, age, disease)] = trend_factor

In [12]:
# Calculate the normalizing factor

def calculate_scaling_factor(train_df, geo_disease_df, prob_df, trend_dict):
    """
    Calculate scaling factor by comparing predicted vs actual 2022 values
    """
    
    scaling_factors = []
    
    # Get all combinations from training data
    for (geo, age, disease), group in train_df.groupby(['Geography','Age_Group','Disease']):
        # Get actual 2022 value
        actual_2022 = group['2022'].values[0]
        
        if actual_2022 == 0:
            continue  # Skip zero values
            
        # Get total disease count for this geography-disease
        total_row = geo_disease_df.loc[(geo_disease_df['Geography']==geo) &
                                       (geo_disease_df['Disease']==disease)]
        if len(total_row) > 0:
            total_count = total_row['Count'].values[0]
        else:
            continue
            
        # Get age probability
        prob_row = prob_df.loc[(prob_df['Disease']==disease) &
                               (prob_df['AgeGroup']==age)]
        if len(prob_row) > 0:
            age_prob = prob_row['Probability'].values[0]
        else:
            continue
            
        # Get trend factor (for 2022, this would be 1 since we're using it as baseline)
        trend_factor = 1.0  # For 2022 comparison
        
        # Calculate what your current method would predict for 2022
        predicted_raw = total_count * age_prob * (trend_factor**n)
        
        if predicted_raw > 0:
            scaling_factor = actual_2022 / predicted_raw
            scaling_factors.append(scaling_factor)
    
    # Use median scaling factor to be robust against outliers
    overall_scaling_factor = np.median(scaling_factors) - 1
    
    return overall_scaling_factor

# Apply the scaling factor to your prediction
scaling_factor = calculate_scaling_factor(train_df, geo_disease_df, prob_df, trend_dict)

scaling_factor

np.float64(7.231903266030139)

In [13]:
geographies = train_df['Geography'].unique()
age_groups = train_df['Age_Group'].unique()
diseases = train_df['Disease'].unique()

pred_list = []

In [14]:
for geo in geographies:
    for disease in diseases:
        # Total disease count with bounds
        total_row = geo_disease_df.loc[(geo_disease_df['Geography']==geo) &
                                       (geo_disease_df['Disease']==disease)]
        if len(total_row) > 0:
            total_count = total_row['Count'].values[0]
            total_lower = total_row['LowerBound'].values[0]
            total_upper = total_row['UpperBound'].values[0]
        else:
            total_count = total_lower = total_upper = 0
        
        # Probabilities for age groups with bounds
        probs = prob_df.loc[prob_df['Disease']==disease, ['AgeGroup','Probability','LowerBound','UpperBound']]
        prob_dict = {row['AgeGroup']:(row['Probability'], row['LowerBound'], row['UpperBound']) 
                     for idx,row in probs.iterrows()}
        
        for age in age_groups:
            age_prob, age_lower, age_upper = prob_dict.get(age, (0,0,0))
            trend_factor = trend_dict.get((geo, age, disease), 1)
            
            pred_val = total_count * age_prob * trend_factor * scaling_factor
            pred_lower = total_lower * age_lower * trend_factor * scaling_factor
            pred_upper = total_upper * age_upper * trend_factor * scaling_factor
            
            pred_list.append({
                'Geography': geo,
                'Age_Group': age,
                'Disease': disease,
                'Predicted2023': pred_val,
                'LowerBound': pred_lower,
                'UpperBound': pred_upper
            })

In [15]:
pred_df = pd.DataFrame(pred_list)

### Step 4. Validate by R-squared values

In [16]:
merged_df = pd.merge(pred_df, test_df, on=['Geography', 'Age_Group', 'Disease'], how='left')  

In [17]:
y_true = merged_df['2023'].values
y_pred = merged_df['Predicted2023'].values

In [18]:
r2 = r2_score(y_true, y_pred)
print("R-squared value: ", round(r2, 4))

R-squared value:  0.9751


In [19]:
merged_df.to_csv('../results/disease_prevalence_pred.csv', index=None)

### Step 5. Let's Forecast it for 2024

In [20]:
pred_list_2024 = []

for geo in geographies:
    for disease in diseases:
        total_row = geo_disease_df.loc[
            (geo_disease_df['Geography'] == geo) &
            (geo_disease_df['Disease'] == disease)
        ]
        if len(total_row) > 0:
            total_count = total_row['Count'].values[0]
            total_lower = total_row['LowerBound'].values[0]
            total_upper = total_row['UpperBound'].values[0]
        else:
            total_count = total_lower = total_upper = 0

        probs = prob_df.loc[
            prob_df['Disease'] == disease, 
            ['AgeGroup', 'Probability', 'LowerBound', 'UpperBound']
        ]
        prob_dict = {row['AgeGroup']:(row['Probability'], row['LowerBound'], row['UpperBound']) 
                     for idx,row in probs.iterrows()}

        for age in age_groups:
            age_prob, age_lower, age_upper = prob_dict.get(age, (0, 0, 0))
            trend_factor = trend_dict.get((geo, age, disease), 1)

            pred_val = total_count * age_prob * (trend_factor**2) * (1.4*scaling_factor)
            pred_lower = total_lower * age_lower * (trend_factor**2) * (1.4*scaling_factor)
            pred_upper = total_upper * age_upper * (trend_factor**2) * (1.4*scaling_factor)

            pred_list_2024.append({
                'Geography': geo,
                'Age_Group': age,
                'Disease': disease,
                'Predicted2024': pred_val,
                'LowerBound': pred_lower,
                'UpperBound': pred_upper
            })

pred_df_2024 = pd.DataFrame(pred_list_2024)

In [21]:
pred_df_2024.to_csv('../results/disease_prevalence_forecast_2024.csv', index=None)

<hr />